In [1]:
import pandas as pd
import json
import requests
import zlib
import requests
import zipfile,io

# 工作描述清理

In [1]:
job_desc = ! cat job.json | sed "s/$(echo -ne $'\u2028')//g"

In [6]:
job_desc[0]

'{"custno":"7c01228e-5bc6-479d-9b18-c95e6678a468","jobno":"4371531","job":"國小安親課輔老師","jobcat1":"2016002008","jobcat2":"2016002011","jobcat3":"0","edu":56,"salary_low":990,"salary_high":990,"role":1,"language1":14444,"language2":1111,"language3":1111,"period":2,"major_cat":"0","major_cat2":"0","major_cat3":"0","industry":"1005001009","worktime":"週休二日","role_status":2049,"s2":4,"s3":0,"addr_no":6001005004,"s9":1,"need_emp":1,"need_emp1":2,"startby":2,"exp_jobcat1":"0","exp_jobcat2":"0","exp_jobcat3":"0","description":"※輔導國小課業內容。\\r\\n※中、英語能力尤佳。\\r\\n需有相關課輔經驗者，無經驗者勿試。","others":"1.提供勞健保\\n2.待遇優\\n3.需有耐心.愛心.抗壓性高/配合度佳-無誠者勿試/有安親經歷者尤佳"}'

In [2]:
len(job_desc)

458638

In [8]:
list_correct_parsed_lines = []
list_wrong_parsed_lines = []
for json_line in job_desc:
    try:
        list_correct_parsed_lines.append(json.loads(json_line))           
    except:
#         print(json_line)
        list_wrong_parsed_lines.append(json_line)

In [9]:
len(list_correct_parsed_lines)

458638

In [10]:
len(list_wrong_parsed_lines)

0

In [11]:
df_correct_lines = pd.DataFrame(list_correct_parsed_lines)

In [14]:
df_correct_lines[0:5]

,addr_no,custno,description,edu,exp_jobcat1,exp_jobcat2,exp_jobcat3,industry,job,jobcat1,...,period,role,role_status,s2,s3,s9,salary_high,salary_low,startby,worktime
0,6001005004,7c01228e-5bc6-479d-9b18-c95e6678a468,※輔導國小課業內容。\r\n※中、英語能力尤佳。\r\n需有相關課輔經驗者，無經驗者勿試。,56,0,0,0,1005001009,國小安親課輔老師,2016002008,...,2,1,2049,4,0,1,990,990,2,週休二日
1,6001005012,6976d5c5-26a5-467d-8c00-eec6cc569c56,環境、空汙、廢水的管理。,60,0,0,0,1002007008,環保專職人員,2009004003,...,-1,1,1,0,0,1,990,990,3,週休二日
2,6001005012,6976d5c5-26a5-467d-8c00-eec6cc569c56,1.負責生產線機器、設備之操作，並維持機台正常運作。\r\n2.進行製造現場的產品組裝、檢驗...,6,0,0,0,1002007008,現場操作人員(固定班),2010001002,...,-1,1,3,0,0,1,990,990,3,依公司規定
3,6001001004,6976d5c5-26a5-467d-8c00-eec6cc569c56,1.產品銷售\r\n2.顧客服務\r\n3.商品管理\r\n4.開發客戶\r\n5.接洽業務,60,0,0,0,1002007008,業務人員(北區),2005003004,...,-1,1,75,0,15,1,990,990,0,依公司規定
4,6001008007,6976d5c5-26a5-467d-8c00-eec6cc569c56,1.產品銷售\r\n2.顧客服務\r\n3.商品管理\r\n4.開發客戶\r\n5.接洽業務,60,0,0,0,1002007008,業務人員(南區),2005003004,...,-1,1,75,0,15,1,990,990,0,依公司規定


# 公司資料

In [23]:
df_company = pd.read_json('./company.json',lines=True,)

In [25]:
df_company = df_company[['name','invoice']]

In [48]:
len(df_company)

340715

In [2]:
res = requests.get('http://www.fia.gov.tw/opendata/bgmopen1.zip')

In [3]:
z = zipfile.ZipFile(io.BytesIO(res.content))
z.extractall('.')

In [15]:
df_invoice_open = pd.read_csv('./BGMOPEN1.csv',usecols=['營業地址','統一編號'])

In [18]:
df_invoice_open = df_invoice_open.dropna()

In [21]:
df_invoice_open['統一編號'] = df_invoice_open['統一編號'].astype('int')

/home/kuo/anaconda3/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
df_company.merge(df_invoice_open,left_on='invoice',right_on='統一編號')

,name,invoice,營業地址,統一編號
0,海邊西子灣餐廳,10207917,高雄市林園區頂厝里０１３鄰沿海路三段２１６號１樓,10207917
1,旅沐豆行,10302172,臺北市中山區江山里錦州街４１６號１樓,10302172
2,(Bella 99 DogSalon)畇安工作坊,10332289,臺北市士林區蘭雅里中山北路６段２８２號２樓,10332289
3,財團法人台北市國際社區服務文教基金會,1061362,臺北市士林區中山北路６段２９０巷２５號１樓,1061362
4,(新毅印刷)新毅企業社,10702064,臺南市北 區成德里育德二路１５１號１樓,10702064
5,金億行銷社,10715981,臺南市北 區大光里和緯路１段６２號１樓,10715981
6,松賀園和風洋食_松賀園企業社,10895267,屏東縣竹田鄉竹田村水源路２之１０號,10895267
7,戰國策網路科技股份有限公司,11002716,臺北市中山區朱園里南京東路２段１５０號１０樓,11002716
8,幸福室內裝修工程有限公司,11428804,臺北市大安區龍生里和平東路２段１７７號９樓,11428804
9,南山人壽─建安獨立區辦公室,11456006,臺北市信義區景新里莊敬路１６８號,11456006


In [36]:
_.to_csv('20180711_公司營業地址(only營業中)_.csv')

In [43]:
df_invoice_close = pd.read_csv('./BGMOPEN1X.csv',usecols=['營業地址','統一編號'])

In [44]:
df_invoice_close = df_invoice_closed.dropna()

In [45]:
df_invoice_close['統一編號'] = df_invoice_close['統一編號'].astype('int')

In [49]:
df_company.merge(df_invoice_close,left_on='invoice',right_on='統一編號')

,name,invoice,營業地址,統一編號
0,群慧有限公司,12898986,新北市新莊區營盤里中正路５１４巷１０５弄１１８號,12898986
1,雲鼎網路有限公司,53084680,臺北市內湖區石潭里南京東路６段４６１號３樓,53084680
2,全富發彩券行,37573161,臺南市善化區東關里中山路４４６號１樓,37573161
3,齊齊小舖,26230512,臺北市大安區古風里泰順街５０巷３６號１樓,26230512
4,花旗蛋糕有限公司,24498253,臺北市大安區龍坡里新生南路３段６之１號,24498253
5,恆輝廣告有限公司,54724024,新北市中和區秀峰里秀峰街１１９巷６弄１８－２號,54724024
6,騰瑪工程有限公司,54380702,金門縣金城鎮東門里民族路３６號,54380702
7,松果室內裝修有限公司,53492980,臺中市西 區藍興里大全街１５４巷４２號１樓,53492980
8,宇禾視覺形象有限公司,24631647,苗栗縣頭份市蟠桃里００５鄰大亨街１５８號１樓,24631647
9,騰日興業有限公司,24829248,臺中市北屯區平興里崇德二路一段８６號,24829248


In [50]:
_.to_csv('20180711_公司營業地址(only歇業中)_.csv')